In [11]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


In [12]:
data = pd.read_csv('jkt_district_2.csv') 

In [13]:
len(data)

42

In [14]:
data.columns=['Borough','Neighborhood','Latitude','Longitude']
data['Borough'] = data['Borough'].str.title() 
data['Neighborhood'] = data['Neighborhood'].str.title() 
data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Jakarta Pusat,Sawah Besar,-6.152958,106.832571
1,Jakarta Pusat,Cempaka Putih,-6.182035,106.866739
2,Jakarta Barat,Kalideres,-6.133608,106.703357
3,Jakarta Timur,Jatinegara,-6.225536,106.873590
4,Jakarta Barat,Tambora,-6.148661,106.803828


In [15]:
# create map of Jakarta using latitude and longitude values
map_jkt = folium.Map(location=[-6.2194891,106.8562831], zoom_start=11)

# add markers to map
for idx,row in data.iterrows():
    label = '{}'.format(row['Neighborhood'])
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row['Latitude'], row['Longitude']],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_jkt)  
    
map_jkt

In [19]:
# save the map as HTML file
map_jkt.save('map_jkt.html')

In [20]:
# define Foursquare Credentials and Version
CLIENT_ID = 'XEDPVZFLOUWNOKEOOBZCP1FL4FZY5RVEOCXNW34RKFKQWUSW' # your Foursquare ID
CLIENT_SECRET = 'X4Q43SQMT3KEGKKK4UBQYRZX4XDF1Y5RPVUDCAFCUWZC0INP' # your Foursquare Secret
VERSION = '20180323' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XEDPVZFLOUWNOKEOOBZCP1FL4FZY5RVEOCXNW34RKFKQWUSW
CLIENT_SECRET:X4Q43SQMT3KEGKKK4UBQYRZX4XDF1Y5RPVUDCAFCUWZC0INP


In [21]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(data['Latitude'], data['Longitude'], data['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [22]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(3453, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Sawah Besar,-6.152958,106.832571,Rumah Makan Medan Baru,-6.158865,106.831763,Indonesian Restaurant
1,Sawah Besar,-6.152958,106.832571,Starbucks - Rs Husada,-6.147282,106.828670,Coffee Shop
2,Sawah Besar,-6.152958,106.832571,Gado Gado Tamansari Raya,-6.152726,106.827043,Salad Place
3,Sawah Besar,-6.152958,106.832571,Fashion hotel,-6.149079,106.835513,Hotel
4,Sawah Besar,-6.152958,106.832571,Hokian Lomie Mikado,-6.159956,106.832026,Chinese Restaurant


In [23]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Cakung,28,28,28,28,28,28
Cempaka Putih,100,100,100,100,100,100
Cengkareng,53,53,53,53,53,53
Cilandak,100,100,100,100,100,100
Cilincing,3,3,3,3,3,3
Cipayung,23,23,23,23,23,23
Ciracas,35,35,35,35,35,35
Duren Sawit,66,66,66,66,66,66
Gambir,100,100,100,100,100,100


In [24]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 245 uniques categories.


In [25]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Indonesian Restaurant', 'Coffee Shop', 'Salad Place', 'Hotel',
       'Chinese Restaurant', 'Karaoke Bar', 'Hotel Bar', 'BBQ Joint',
       'Dessert Shop', 'Noodle House', 'Food Truck', 'Concert Hall',
       'Indonesian Meatball Place', 'Donut Shop', 'Restaurant', 'Theater',
       'Office', 'Sandwich Place', 'Asian Restaurant', 'Art Gallery',
       'Diner', 'Seafood Restaurant', 'Steakhouse', 'Nightclub',
       'Japanese Restaurant', 'Pizza Place', 'Bakery', 'Breakfast Spot',
       'Supermarket', 'Sundanese Restaurant', 'American Restaurant',
       'Cantonese Restaurant', 'Soup Place', 'Racetrack', 'Bookstore',
       'Dim Sum Restaurant', 'Padangnese Restaurant',
       'Sporting Goods Shop', 'Fried Chicken Joint', 'Hardware Store',
       'Snack Place', 'Pharmacy', 'Multiplex', 'Moroccan Restaurant',
       'Bistro', 'Café', 'Korean Restaurant', 'Shopping Mall',
       'Acehnese Restaurant', 'Farmers Market'], dtype=object)

In [26]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

In [27]:
# one hot encoding
jkt_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
jkt_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [jkt_onehot.columns[-1]] + list(jkt_onehot.columns[:-1])
jkt_onehot = jkt_onehot[fixed_columns]

print(jkt_onehot.shape)
jkt_onehot.head()

(3453, 246)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Balinese Restaurant,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Garden,Betawinese Restaurant,Big Box Store,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Butcher,Cafeteria,Café,Camera Store,Campground,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Cafeteria,College Residence Hall,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Fabric Shop,Factory,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Malay Restaurant,Manadonese Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Other Great Outdoors,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Student Center,Sundanese Restaurant,Supermarket,Surf Spot,Sushi Restaurant,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train,Train Station,Travel Agency,Turkish Restaurant,Udon Restaurant,University,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Water Park,Wine Bar,Winery,Wings Joint,Women's Store
0,Sawah Besar,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [28]:
jkt_grouped = jkt_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(jkt_grouped.shape)
jkt_grouped

(42, 246)


,Neighborhoods,Accessories Store,Acehnese Restaurant,African Restaurant,Airport,Airport Lounge,Airport Terminal,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Australian Restaurant,Auto Dealership,Auto Workshop,Automotive Shop,BBQ Joint,Badminton Court,Bakery,Balinese Restaurant,Bank,Bar,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Garden,Betawinese Restaurant,Big Box Store,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Bridal Shop,Bubble Tea Shop,Buffet,Building,Burger Joint,Butcher,Cafeteria,Café,Camera Store,Campground,Cantonese Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Coffee Shop,College Cafeteria,College Residence Hall,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Creperie,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Doctor's Office,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Elementary School,Fabric Shop,Factory,Farmers Market,Fast Food Restaurant,Field,Fish & Chips Shop,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Hardware Store,Health & Beauty Service,High School,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Housing Development,Ice Cream Shop,Indian Restaurant,Indonesian Meatball Place,Indonesian Restaurant,Italian Restaurant,Japanese Restaurant,Javanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Korean Restaurant,Lake,Lounge,Malay Restaurant,Manadonese Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Neighborhood,Nightclub,Noodle House,Office,Other Great Outdoors,Padangnese Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Store,Pharmacy,Photography Studio,Piano Bar,Pier,Pizza Place,Playground,Plaza,Pool,Pool Hall,Pub,Public Art,Racetrack,Ramen Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,River,Salad Place,Salon / Barbershop,Sandwich Place,Satay Restaurant,Seafood Restaurant,Shabu-Shabu Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Street Food Gathering,Student Center,Sundanese Restaurant,Supermarket,Surf Spot,Sushi Restaurant,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Toy / Game Store,Trail,Train,Train Station,Travel Agency,Turkish Restaurant,Udon Restaurant,University,Vacation Rental,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Water Park,Wine Bar,Winery,Wings Joint,Women's Store
0,Cakung,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.035714,0.000000,0.00,0.00,0.00000,0.000000,0.035714,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.00000,0.00,0.00000,0.000000,0.000000,0.00,0.00,0.035714,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.035714,0.000000,0.00,0.00,0.00,0.000000,0.035714,0.00,0.000000,0.035714,0.035714,0.00,0.000000,0.000000,0.000000,0.000000,0.0

In [29]:
len(jkt_grouped[jkt_grouped["Shopping Mall"] > 0])

20

In [30]:
jkt_mall = jkt_grouped[["Neighborhoods","Shopping Mall"]]

In [31]:
jkt_mall.head()

,Neighborhoods,Shopping Mall
0,Cakung,0.00
1,Cempaka Putih,0.01
2,Cengkareng,0.00
3,Cilandak,0.01
4,Cilincing,0.00


In [32]:
# set number of clusters
kclusters = 3

jkt_clustering = jkt_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(jkt_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 2, 0, 0, 1, 0, 0, 1])

In [33]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
jkt_merged = jkt_mall.copy()

# add clustering labels
jkt_merged["Cluster Labels"] = kmeans.labels_

In [34]:
jkt_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
jkt_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Cakung,0.00,0
1,Cempaka Putih,0.01,2
2,Cengkareng,0.00,0
3,Cilandak,0.01,2
4,Cilincing,0.00,0


In [35]:
# merge jkt_grouped with jkt_data to add latitude/longitude for each neighborhood
jkt_merged = jkt_merged.join(data.set_index("Neighborhood"), on="Neighborhood")
del jkt_merged['Borough']
print(jkt_merged.shape)
jkt_merged.head() # check the last columns!

(42, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Cakung,0.00,0,-6.189808,106.939408
1,Cempaka Putih,0.01,2,-6.182035,106.866739
2,Cengkareng,0.00,0,-6.151473,106.737443
3,Cilandak,0.01,2,-6.288974,106.794213
4,Cilincing,0.00,0,-6.123956,106.938039


In [36]:
# sort the results by Cluster Labels
print(jkt_merged.shape)
jkt_merged.sort_values(["Cluster Labels"], inplace=True)
jkt_merged

(42, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Cakung,0.000000,0,-6.189808,106.939408
40,Tanjung Priok,0.000000,0,-6.128501,106.877419
29,Pasar Minggu,0.000000,0,-6.290114,106.831325
30,Pasar Rebo,0.000000,0,-6.325960,106.855982
31,Penjaringan,0.000000,0,-6.120833,106.774103
16,Kebon Jeruk,0.000000,0,-6.194432,106.769319
32,Pesanggrahan,0.000000,0,-6.246957,106.758036
14,Kebayoran Baru,0.000000,0,-6.243976,106.802370
13,Kalideres,0.000000,0,-6.133608,106.703357
12,Johar Baru,0.000000,0,-6.180202,106.854249


In [40]:
# create map
map_clusters = folium.Map(location=[-6.2194891,106.8562831], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(jkt_merged['Latitude'], jkt_merged['Longitude'], jkt_merged['Neighborhood'], jkt_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [41]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

In [42]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Cakung,0.0,0,-6.189808,106.939408
40,Tanjung Priok,0.0,0,-6.128501,106.877419
29,Pasar Minggu,0.0,0,-6.290114,106.831325
30,Pasar Rebo,0.0,0,-6.325960,106.855982
31,Penjaringan,0.0,0,-6.120833,106.774103
16,Kebon Jeruk,0.0,0,-6.194432,106.769319
32,Pesanggrahan,0.0,0,-6.246957,106.758036
14,Kebayoran Baru,0.0,0,-6.243976,106.802370
13,Kalideres,0.0,0,-6.133608,106.703357
12,Johar Baru,0.0,0,-6.180202,106.854249


In [44]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
36,Setiabudi,0.030000,1,-6.216270,106.830222
27,Pal Merah,0.040000,1,-6.189002,106.798577
20,Koja,0.055556,1,-6.121772,106.904721
21,Kramatjati,0.037736,1,-6.276859,106.865906
6,Ciracas,0.028571,1,-6.329403,106.877352
17,Kelapa Gading,0.040000,1,-6.162517,106.904483
9,Grogol Petamburan,0.030000,1,-6.164118,106.787194


In [43]:
jkt_merged.loc[jkt_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
25,Menteng,0.020000,2,-6.194293,106.836251
39,Tanah Abang,0.020000,2,-6.200213,106.811674
1,Cempaka Putih,0.010000,2,-6.182035,106.866739
3,Cilandak,0.010000,2,-6.288974,106.794213
11,Jatinegara,0.020000,2,-6.225536,106.873590
33,Pulogadung,0.010000,2,-6.195944,106.890631
15,Kebayoran Lama,0.010000,2,-6.243439,106.779442
18,Kemayoran,0.011628,2,-6.163641,106.855970
19,Kembangan,0.020000,2,-6.198578,106.741323
28,Pancoran,0.020619,2,-6.253196,106.846305
